# TS8: Filtrado Digital Lineal de ECG

**Materia:** Análisis y Procesamiento de Señales  
**Carrera:** Ingeniería Electrónica – UNSAM  
**Docente:** Mariano Llamedo Soria, Francisco Hernan Ledesma y David Ezequiel Benoit  
**Integrantes:** Naudeau Nahiara y Gil María Serena  
**Fecha:** 16/11/2025  

---

## Objetivo

El objetivo de esta tarea es diseñar y aplicar filtros digitales lineales para mejorar la calidad de un registro de ECG real.  
Se busca:

- eliminar el movimiento de la línea de base (baja frecuencia),  
- reducir el ruido muscular y los artefactos de electrodos (alta frecuencia),  
- y preservar la morfología de los latidos cardíacos.

Para ello se diseñarán filtros FIR e IIR y se comparará su desempeño, verificando que la señal filtrada quede más limpia, suave y con una línea de base estable.

---

## Introducción

El electrocardiograma (ECG) es una señal que registra la actividad eléctrica del corazón y permite identificar la presencia de latidos normales o patológicos.  
Sin embargo, en registros reales —especialmente durante una prueba de esfuerzo— la señal suele estar afectada por distintos tipos de ruido:

- **Movimiento de electrodos:** cambios bruscos en el contacto piel–electrodo, que generan picos y artefactos de alta amplitud.
- **Ruido muscular (EMG):** componentes de alta frecuencia originadas por la actividad de los músculos cercanos.
- **Movimiento de la línea de base:** variaciones lentas producidas por la respiración u otros desplazamientos del cuerpo.

Estas interferencias pueden dificultar la correcta lectura de la señal y alterar la forma del complejo QRS, las ondas P y T, o el nivel de línea de base.  
Por este motivo se emplea **filtrado digital lineal**, que permite atenuar selectivamente las frecuencias indeseables mientras se preserva la información útil del ECG.

En esta tarea se trabajará con el archivo `ecg.mat`, que contiene un registro real muestreado a  
$$ f_s = 1\ \text{kHz}, $$
junto con latidos promedio (normal y ventricular) que servirán como referencia para evaluar la calidad del filtrado.  
A partir de estas señales, se diseñarán filtros FIR e IIR basados en una plantilla de diseño ajustada a la banda de frecuencias característica del ECG, y se analizará su desempeño en regiones con y sin ruido.

----